# OPTIMIZING HYPERPARAMETERS - CNN MODEL

## CONTENTS:

1. Import Libraries and Data¶
2. Data Wrangling and Reshaping
3. Data Splitting
4. Bayesian Hyperparameter Optimization
5. Compiling and Running CNN Model with optimized search parameters
6. Confusion Matrix Creation


### 1. IMPORT LIBRARIES AND DATA

In [384]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
from sklearn.multioutput import MultiOutputClassifier
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [385]:
# Define path for where data is stored
path = r'C:\Users\Administrator\Documents\data analytics\Machine Learning\ClimateWins\Data Sets'

In [386]:
# Import the weather data that was already scaled. 
climate = pd.read_csv(os.path.join(path, 'data_scaled.csv'))

In [387]:
climate.head(3)

,Unnamed: 0,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,0,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,...,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,1,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,...,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,2,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,...,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [388]:
# Drop the unnecessary unnamed column
climate= climate.drop('Unnamed: 0', axis = 1)

In [389]:
climate.head(3)

,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,...,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,-0.528623,...,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,-0.582946,...,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,-0.257010,...,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [390]:
climate.shape

(22950, 170)

In [391]:
# Read in the Answers data.
answers = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [392]:
answers.shape

(22950, 16)

### 2. DATA WRANGLING AND RESHAPING

In [393]:
# Drop Gdansk, Roma and Tours since they are missing from the Answers file. I use a filtering function. 

# List of city names
cities = ['GDANSK', 'ROMA', 'TOURS']

# Find columns that include any of the city names
cols_to_drop = [col for col in climate.columns if any(city in col for city in cities)]

# Drop those columns
climatedr = climate.drop(columns=cols_to_drop)

# Count nr of columns in climatedr
num_columns = climatedr.shape[1]
print(num_columns)

149


In [394]:
climatedr.shape

(22950, 149)

In [395]:
climatedr.head(3)

,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,...,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,-0.528623,...,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,-0.582946,...,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,-0.257010,...,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [396]:
climatedr.isnull().sum()

MONTH                 0
BASEL_cloud_cover     0
BASEL_wind_speed      0
BASEL_humidity        0
BASEL_pressure        0
                     ..
VALENTIA_sunshine     0
VALENTIA_temp_mean    0
VALENTIA_temp_min     0
VALENTIA_temp_max     0
DATE                  0
Length: 149, dtype: int64

In [397]:
# Extract the different observation types

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [398]:
# Create a dictionary to store the count of stations for each observation type
station_counts = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in climatedr.columns if col.endswith(obs)]
    
    # Count the number of stations (i.e., the number of columns) for the current observation type
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type
print("Number of stations covered by each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")


Number of stations covered by each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


Windspeed and snowdepth are missing entries and will be dropped. 

In [399]:
# Get a list of columns containing 'wind_speed' or 'snow_depth'
cols_to_drop = [col for col in climatedr.columns if '_wind_speed' in col or '_snow_depth' in col]

# Drop the columns
climate2 = climatedr.drop(cols_to_drop, axis=1)

In [400]:
climate2.shape # 15 columns less since they were dropped.

(22950, 134)

One entry missing for cloud humidity, pression and cloud cover. 

In [401]:
# Find the stations with the above entries missing
# Get all column names
all_columns = climate2.columns.tolist()
# Exclude 'DATE' and 'MONTH' columns
all_columns = [col for col in all_columns if col not in ['DATE', 'MONTH']]  
# Extract unique weather station names
weather_stations = set()  # Use a set to automatically store only unique values
for col in all_columns:
    station_name = col.split('_')[0]  # Split the column name at the underscore and take the first part
    weather_stations.add(station_name)

# Print the list of weather stations
print(weather_stations)

{'KASSEL', 'MAASTRICHT', 'BELGRADE', 'MADRID', 'STOCKHOLM', 'VALENTIA', 'BASEL', 'HEATHROW', 'LJUBLJANA', 'SONNBLICK', 'DEBILT', 'DUSSELDORF', 'BUDAPEST', 'MUNCHENB', 'OSLO'}


In [402]:
# Find stations missing observation types
observation_types = ['cloud_cover', 'humidity', 'pressure']

missing_stations_by_observation = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in climate2.columns if col.endswith(obs)]
    
    # Extract station names by removing the observation type from the column names
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    
    # Identify stations that are in all_stations but missing from the current observation type
    missing_stations = weather_stations - station_names
    
    # Store the missing station names in the dictionary
    missing_stations_by_observation[obs] = missing_stations

# Print the missing station names for each observation type
for obs, missing_stations in missing_stations_by_observation.items():
    print(f"\nStations missing from {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Stations missing from cloud_cover:
KASSEL

Stations missing from humidity:
STOCKHOLM

Stations missing from pressure:
MUNCHENB


In [403]:
climate2.head(3)

,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,-1.599964,0.660514,0.826097,-0.001949,-1.101066,-0.265148,-0.902918,-0.528623,-0.845652,-0.478356,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,-1.599964,0.244897,0.735760,-0.001949,-1.058108,1.658760,-0.810126,-0.582946,-0.462450,-0.569988,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,-1.599964,1.076130,1.277781,-0.001949,-1.251420,0.155707,-1.065304,-0.257010,-0.186545,-0.592896,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [404]:
# Cloud cover is the start of a stations data, Kassel is next to Heathrow, find the position of Heathrow_temp_max for the insertion of Kassel_cloud_cover
climate2.columns.get_loc('HEATHROW_temp_max')

54

In [405]:
# Find the position for insertion of Stockholm humidity
climate2.columns.get_loc('STOCKHOLM_cloud_cover') #humidity is 1 after cloud cover so (result +1)

116

In [406]:
# Find position for Munchenb pressure
climate2.columns.get_loc('MUNCHENB_cloud_cover') # pressure is 2 after cloud cover so (result +2)

90

In [407]:
# Insert new columns into "unscaled" at specific positions.
# The data for these new columns is taken from weather stations they are close to

climate2.insert(56,'KASSEL_cloud_cover', climate2['DUSSELDORF_cloud_cover'])
climate2.insert(119, 'STOCKHOLM_humidity', climate2['OSLO_humidity'])
climate2.insert(94,'MUNCHENB_pressure',climate2['BASEL_pressure'])

In [408]:
climate2.columns.tolist()

['MONTH',
 'BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_te

In [409]:
climate2.shape

(22950, 137)

In [410]:
# Drop unnecessary columns
climate2.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [411]:
# confirm drop
climate2.shape

(22950, 135)

In [412]:
answers.head(3)

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [413]:
# drop unneeded column from second dataset
answers.drop(columns = 'DATE', inplace = True)

In [414]:
# check drop
answers.head(3)

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [415]:
answers.shape

(22950, 15)

In [416]:
# Export cleaned dataset
climate2.to_csv(os.path.join(path, 'climate_clean.csv'), index=False)

In [417]:
# Create an 'X' matrix by reloading and naming our data 'X'
x=pd.read_csv(os.path.join(path, 'climate_clean.csv'), index_col=False)

In [418]:
x.head(3)

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,0.660514,0.826097,-0.001949,-1.101066,-0.265148,-0.902918,-0.528623,-0.845652,-0.478356,-1.206433,...,-0.639538,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,0.372147,-0.668215,-0.519743,-0.752237
1,0.244897,0.735760,-0.001949,-1.058108,1.658760,-0.810126,-0.582946,-0.462450,-0.569988,0.652846,...,-0.628550,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.829285,-0.548046,-0.629054,-0.407141
2,1.076130,1.277781,-0.001949,-1.251420,0.155707,-1.065304,-0.257010,-0.186545,-0.592896,0.652846,...,-0.727444,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-1.009500,-0.067372,0.054135,-0.177078


In [419]:
y = answers

In [420]:
x.shape

(22950, 135)

In [421]:
# Turn X and y into arrays
x = np.array(x)
y = np.array(y)
x

array([[ 6.60513663e-01,  8.26096599e-01, -1.94863388e-03, ...,
        -6.68214979e-01, -5.19743407e-01, -7.52236990e-01],
       [ 2.44896945e-01,  7.35759689e-01, -1.94863388e-03, ...,
        -5.48046319e-01, -6.29053523e-01, -4.07141387e-01],
       [ 1.07613038e+00,  1.27778115e+00, -1.94863388e-03, ...,
        -6.73716818e-02,  5.41347039e-02, -1.77077651e-01],
       ...,
       [-5.86336492e-01,  1.30644098e-02,  7.16401992e-01, ...,
        -7.28735214e-03, -5.20354258e-04, -4.52984969e-03],
       [-1.70719774e-01,  3.74412049e-01,  4.87141154e-01, ...,
        -7.28735214e-03, -5.20354258e-04, -4.52984969e-03],
       [-1.70719774e-01,  7.35759689e-01,  1.96744092e-01, ...,
        -7.28735214e-03, -5.20354258e-04, -4.52984969e-03]])

In [422]:
x = x.reshape(-1,15,9)

In [423]:
# Verify Shape
x.shape

(22950, 15, 9)

In [433]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [434]:
# Verify Shape
y.shape

(22950,)

In [435]:
x

array([[[ 6.60513663e-01,  8.26096599e-01, -1.94863388e-03, ...,
         -5.28623012e-01, -8.45651922e-01, -4.78356271e-01],
        [-1.20643263e+00,  9.05270489e-01,  3.21770762e-01, ...,
         -1.01687613e+00, -1.22021042e+00, -9.49202784e-01],
        [-2.55240242e-01, -1.00976762e-02, -5.87602906e-03, ...,
         -1.09916317e+00, -1.11943125e+00, -1.13683915e+00],
        ...,
        [-4.28835402e-01, -6.29430381e-01,  9.52340493e-02, ...,
         -1.24330511e-01, -1.70125111e-01, -6.33220094e-02],
        [-6.01023076e-02, -3.46465175e-03,  1.54402321e+00, ...,
         -3.91072163e-01, -2.90438548e-01, -6.39537957e-01],
        [-4.43700743e-01,  7.61754381e-01, -1.29974368e+00, ...,
         -6.68214979e-01, -5.19743407e-01, -7.52236990e-01]],

       [[ 2.44896945e-01,  7.35759689e-01, -1.94863388e-03, ...,
         -5.82945633e-01, -4.62450189e-01, -5.69988328e-01],
        [ 6.52845676e-01,  1.11911758e+00,  1.68378867e-02, ...,
         -1.10766931e+00, -8.18701592e

In [436]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

### 3. DATA SPLITTING

In [441]:
# Split data into train and test sets

x_train, x_test, y_train, y_test = train_test_split(x,y,random_state = 42)

In [442]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


### 4. BAYESIAN MODEL OPTIMIZATION

In [444]:
timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [445]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, x_train, y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [446]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/22


C:\Users\Administrator\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


43/43 - 1s - 31ms/step - accuracy: 0.6438 - loss: nan
Epoch 2/22
43/43 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/22
43/43 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/22
43/43 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/22
43/43 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 16/22
43/43 - 0s - 7ms/step - accuracy

C:\Users\Administrator\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


43/43 - 1s - 29ms/step - accuracy: 0.6357 - loss: nan
Epoch 2/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/22
43/43 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/22
43/43 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/22
43/43 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 16/22
43/43 - 0s - 6ms/step - accuracy

...

C:\Users\Administrator\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 - 1s - 16ms/step - accuracy: 0.4928 - loss: 1.9161
Epoch 2/39
37/37 - 0s - 4ms/step - accuracy: 0.6349 - loss: 1.2247
Epoch 3/39
37/37 - 0s - 4ms/step - accuracy: 0.6390 - loss: 1.1284
Epoch 4/39
37/37 - 0s - 4ms/step - accuracy: 0.6434 - loss: 1.0798
Epoch 5/39
37/37 - 0s - 4ms/step - accuracy: 0.6522 - loss: 1.0477
Epoch 6/39
37/37 - 0s - 5ms/step - accuracy: 0.6593 - loss: 1.0219
Epoch 7/39
37/37 - 0s - 4ms/step - accuracy: 0.6632 - loss: 1.0033
Epoch 8/39
37/37 - 0s - 4ms/step - accuracy: 0.6651 - loss: 0.9887
Epoch 9/39
37/37 - 0s - 4ms/step - accuracy: 0.6715 - loss: 0.9746
Epoch 10/39
37/37 - 0s - 5ms/step - accuracy: 0.6720 - loss: 0.9633
Epoch 11/39
37/37 - 0s - 4ms/step - accuracy: 0.6760 - loss: 0.9524
Epoch 12/39
37/37 - 0s - 5ms/step - accuracy: 0.6787 - loss: 0.9418
Epoch 13/39
37/37 - 0s - 4ms/step - accuracy: 0.6819 - loss: 0.9336
Epoch 14/39
37/37 - 0s - 4ms/step - accuracy: 0.6825 - loss: 0.9261
Epoch 15/39
37/37 - 0s - 4ms/step - accuracy: 0.6832 - loss: 0.9180

C:\Users\Administrator\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 - 1s - 16ms/step - accuracy: 0.4877 - loss: 1.8253
Epoch 2/39
37/37 - 0s - 4ms/step - accuracy: 0.6545 - loss: 1.1696
Epoch 3/39
37/37 - 0s - 4ms/step - accuracy: 0.6654 - loss: 1.0814
Epoch 4/39
37/37 - 0s - 4ms/step - accuracy: 0.6754 - loss: 1.0458
Epoch 5/39
37/37 - 0s - 4ms/step - accuracy: 0.6805 - loss: 1.0197
Epoch 6/39
37/37 - 0s - 4ms/step - accuracy: 0.6876 - loss: 0.9989
Epoch 7/39
37/37 - 0s - 4ms/step - accuracy: 0.6919 - loss: 0.9809
Epoch 8/39
37/37 - 0s - 4ms/step - accuracy: 0.6951 - loss: 0.9653
Epoch 9/39
37/37 - 0s - 4ms/step - accuracy: 0.6961 - loss: 0.9535
Epoch 10/39
37/37 - 0s - 4ms/step - accuracy: 0.7000 - loss: 0.9407
Epoch 11/39
37/37 - 0s - 4ms/step - accuracy: 0.7000 - loss: 0.9303
Epoch 12/39
37/37 - 0s - 4ms/step - accuracy: 0.7009 - loss: 0.9198
Epoch 13/39
37/37 - 0s - 4ms/step - accuracy: 0.7019 - loss: 0.9108
Epoch 14/39
37/37 - 0s - 4ms/step - accuracy: 0.7028 - loss: 0.9045
Epoch 15/39
37/37 - 0s - 4ms/step - accuracy: 0.7058 - loss: 0.8929

ValueError: Input y contains NaN.

In [ ]:
# More outputs
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

### 4. CNN with optimized search parameters

In [448]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  # Rounded kernel size for Conv1D
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
optimizer = Adadelta(learning_rate=0.7631771981307285)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [449]:
model.summary()

Model: "sequential_84"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_77 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_410 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_411 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_412 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_77 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_77 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_413 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [450]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train, num_classes=15)

In [451]:
# Check shapes

print(f'x_train shape: {x_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

x_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [452]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [453]:
# Fit the model to the data

model.fit(x_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 2s - 44ms/step - accuracy: 0.6293 - loss: 1.4063
Epoch 2/47
38/38 - 0s - 11ms/step - accuracy: 0.7036 - loss: 0.9477
Epoch 3/47
38/38 - 0s - 11ms/step - accuracy: 0.7246 - loss: 0.8592
Epoch 4/47
38/38 - 0s - 10ms/step - accuracy: 0.7441 - loss: 0.7798
Epoch 5/47
38/38 - 0s - 10ms/step - accuracy: 0.7603 - loss: 0.7129
Epoch 6/47
38/38 - 0s - 10ms/step - accuracy: 0.7745 - loss: 0.6649
Epoch 7/47
38/38 - 0s - 10ms/step - accuracy: 0.7876 - loss: 0.6251
Epoch 8/47
38/38 - 0s - 10ms/step - accuracy: 0.7966 - loss: 0.5996
Epoch 9/47
38/38 - 0s - 12ms/step - accuracy: 0.8026 - loss: 0.5778
Epoch 10/47
38/38 - 0s - 10ms/step - accuracy: 0.8103 - loss: 0.5554
Epoch 11/47
38/38 - 0s - 10ms/step - accuracy: 0.8154 - loss: 0.5394
Epoch 12/47
38/38 - 0s - 11ms/step - accuracy: 0.8182 - loss: 0.5219
Epoch 13/47
38/38 - 0s - 11ms/step - accuracy: 0.8248 - loss: 0.5060
Epoch 14/47
38/38 - 0s - 12ms/step - accuracy: 0.8305 - loss: 0.4938
Epoch 15/47
38/38 - 0s - 11ms/step - accura

### 5. CONFUSION MATRIX CREATION

In [457]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [460]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [467]:
# Evaluate
y_pred = model.predict(x_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [469]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3539        81        11       4           3         6       0   
BELGRADE      113       961         7       3           0         1       0   
BUDAPEST       17        59       127       1           0         2       0   
DEBILT         11         8        10      50           0         3       0   
DUSSELDORF      5         2         4       7           6         5       0   
HEATHROW       10         5         3       2           3        56       0   
KASSEL          4         1         1       0           1         1       2   
LJUBLJANA      13         3         3       0           1         1       0   
MAASTRICHT      8         0         0       0           0         1       0   
MADRID         62        22        10       1           0        18       0   
MUNCHENB        7         0         0       0       

# 